In [14]:
%pylab
from numba import njit

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


/home/mike/.conda/envs/idp/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


First we'll write some general-purpose integration functions. Care must be taken to make sure that the correct function evaluation points are being computed.

In [31]:
def Midpoint(func, a, b, N=10):
    H = (b - a)/N
    midpoints = a + H * (0.5 + np.arange(N))
    f = func(midpoints)
    return (H * f).sum()

def LeftPoint(func, a, b, N=10):
    H = (b-a)/N
    leftpoints = np.arange(0,b,H)
    return (H*f(leftpoints)).sum()

def Trapz(func, a, b, N=10):
    H = (b-a)/N
    x = np.linspace(a,b,N+1) # if we have N intervals, we have N+1 abcissae
    f = func(x)
    return np.sum(H * 0.5 * (f[1:] + f[:-1]))

def Simps(func, a, b, N=10):
    H = 2*(b-a)/N # factor of 2 because in the text H is the distance from a to b
    x = np.linspace(a,b,N+1) # if we have N intervals, we have N+1 abcissae
    f = func(x)
    return H/6 * (f[0] + f[-1] + 2*f[2:-2:2].sum() + 4*f[1::2].sum()) # extended Simpson rule - note that I have 

# Doubles the number of evaluation points until we get the error we want. 
# This tries to be fancy by saving the function values from the last iteration
def AdaptiveSimps(func, a, b, errtol=1e-15):
    err = 1e100
    old_integral = 1e100
    N = 2
    f = np.empty(0)
    while err > errtol:
        H = 2*(b-a)/N # factor of 2 because in the text H is the distance from a to b
        x = np.linspace(a,b,N+1) # if we have N intervals, we have N+1 abcissae
        if len(f) > 0:
            fnew = np.empty(N+1) # create a new array to store the points
            fnew[::2] = f  # even values are the old points
            fnew[1::2] = func(x[1::2]) # odd values are the new founds
            f = fnew
        else:
            f = func(x)
        integral = H/6 * (f[0] + f[-1] + 2*f[2:-2:2].sum() + 4*f[1::2].sum())
        err = abs((old_integral - integral)/integral)
        old_integral = integral
        N = N << 1 # double N
    return integral

Now let's do the convergence plot

In [19]:
methods = {"Midpoint": Midpoint, "Trapezoidal": Trapz, "Simpson": Simps, "Left Point": LeftPoint}
plt.clf()

f = lambda x: np.exp(x)
# Note: if you haven't run into this operator, it's a bit shift - we take the integer 1 and shift its bits 
# to the left by the values in arange, which gives us powers of 2. This is the most efficient way to get powers of 2.
npoints = 1 << np.arange(1,28) 
for m in methods:
    method = methods[m]
    errors = np.abs(np.array([method(f, 0, 1, N) for N in npoints]) - 1.718281828459045235)
    plt.scatter(npoints, errors, label=m)
plt.loglog()
plt.ylim(1e-17,1)
plt.xlim(1,2**30)
plt.xlabel(r"Func Evaluations")
plt.ylabel(r"Error")
plt.legend()
plt.savefig("Convergence.pdf", bbox_inches='tight')

We can test our adaptive Simpson function and benchmark it vs. quad

In [33]:
print([AdaptiveSimps(f,0,1,errtol = a) - (np.e-1) for a in np.logspace(-7,-15,9)])
%timeit AdaptiveSimps(f, 0, 1)

[9.102726350462831e-09, 5.689702042843692e-10, 3.556155370176839e-11, 2.2226664952995634e-12, 1.389999226830696e-13, 8.659739592076221e-15, 8.659739592076221e-15, 6.661338147750939e-16, 0.0]
336 µs ± 4.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [36]:
from scipy.integrate import quad
%timeit quad(f, 0, 1, epsrel=1e-15)

14.6 µs ± 29.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
